In [10]:
import os
import json
import geojson
import pandas as pd
from tqdm import tqdm
from osgeo import gdal
from sklearn.model_selection import train_test_split

# custom functions
import sys
sys.path.append('../')
from utils.functions import grab_certain_file
# TODO delete functions.py in data_preperation folder. Use main utils instead.

'''
Read NSO tiles and annotations geojsons, convert lat/lon of tile to pixel coordinates and save pixel coordinates into
.json file If more than on .json file can be saved as one via_regions.json.
Source:  https://github.com/rl02898/detectron2-spacenet. JDP Edits:some and saving json with origins of tile in jsons to allow stiching back
of the tiles.
'''

In [11]:
# Divide dataset and set a random seed for reproducibility of the splits for next script
# TODO: path nso_train and nso_geojson to tiling.py output => Better move output of tiling to those 2 folder in NSO/ 

RANDOM_SEED = 560


# Change name to dataset...
nso_train = "../NSO/train"
nso_test = "../NSO/test"
nso_val = "../NSO/val"
nso_geojson = "../NSO/geojsons"

In [15]:
RANDOM_SEED = 560

nso_small_tiles = "../NSO/NSO_small_tiles"
nso_geojson = "../NSO/geojsons"

# Create JSONs for Detectron2 NO test set
#nso_images = grab_certain_file(".tif", nso_small_tiles)
#train, val = train_test_split(nso_images, test_size=0.2, random_state=RANDOM_SEED)

# Create JSONs for Detectron2 WITH test set
nso_images = grab_certain_file(".tif", nso_small_tiles)
train, test = train_test_split(nso_images, test_size=0.20, random_state=RANDOM_SEED)
train, val = train_test_split(train, test_size=0.25, random_state=RANDOM_SEED)

In [18]:
# Training set

# Create an empty dictionary to store the training set of annotations and their pixel coordinates
train_dict = {}

# Loop over each image in the training set
for file in tqdm(train, desc="Creating JSONs for Detectron2 on train", ncols=150, bar_format="{l_bar}{bar:10}{r_bar}"):
    file_path = os.path.join(nso_small_tiles, file)
    img_id = file.split(".tif")[0]
    geojson_path = os.path.join(nso_geojson, f"{img_id}.geojson")
    
    #Not all tiles have annotations, thus:
    if os.path.exists(geojson_path):
        
        # Load the geojson in gj
        with open(geojson_path) as f:
            gj = geojson.load(f)
        
        # Create a dictionary to store the regions (annotations spatial features) for the image
        regions = {}
        num_buildings = len(gj["features"])
        print (num_buildings) 
        # Open the image with gdal to get pixel size and origin if feature exists
        if num_buildings > 0:
            gdal_image = gdal.Open(file_path)
            
            # Get the pixel width and height(0.5 for nso) and the origin coordinates
            #https://www.gis.usu.edu/~chrisg/python/2009/lectures/ospy_slides4.pdf
            pixel_width, pixel_height = gdal_image.GetGeoTransform()[1], gdal_image.GetGeoTransform()[5]
            originX, originY = gdal_image.GetGeoTransform()[0], gdal_image.GetGeoTransform()[3]
            
            # Loop over each building/assets in the image
            for i in range(num_buildings):
                
                # Get the polygon points for the asset
                #https://stackoverflow.com/questions/23306653/python-accessing-nested-json-data
                points = gj["features"][i]["geometry"]["coordinates"][0]
                
                # If there is only one point, unwarp it=>check
                if len(points) == 1:
                    points = points[0]

                #Empty lists to store pixel coordinates
                all_points_x, all_points_y = [], []
                
                # Convert the lat/long points to pixel coordinates by substacting origin
                for j in range(len(points)):
                    all_points_x.append(int(round((points[j][0] - originX) / pixel_width)))
                    all_points_y.append(int(round((points[j][1] - originY) / pixel_height)))
                    
                # Create a dictionary to store the asset footprint
                regions[str(i)] = {"shape_attributes":
                                       {"name": "polygon",
                                        "all_points_x": all_points_x,
                                        "all_points_y": all_points_y,
                                        "category": 0
                                       },
                                   "region_attributes": {}
                                  }
                print (regions)
        #Should probably save origin x and y here but we still have the og tiles and imgid and allow to stich tiles back together
        #TODO: same for tiles without annot.Eg create json with empty regions in the else
        dictionary = {"file_ref": '',
                      "size": os.path.getsize(file_path),
                      "filename": file.replace(".tif", ".png"),
                      "base64_img_data": '',
                      "file_attributes": {},
                      "regions": regions,
                      "origin_x": originX,
                      "origin_y": originY
                     }
        print (dictionary)
        train_dict[file.replace(".tif", ".png")] = dictionary
    else:
        continue

with open("../NSO/train/nso.json", "w") as f:
    json.dump(train_dict, f)

Creating JSONs for Detectron2 on train:   1%|          | 151/22719 [00:00<00:20, 1086.69it/s]

2
{'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [380, 380, 331, 317, 332, 117, 106, 93, 103, 0, 0, 380], 'all_points_y': [412, 414, 568, 566, 516, 442, 483, 471, 437, 398, 269, 412], 'category': 0}, 'region_attributes': {}}}
{'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [380, 380, 331, 317, 332, 117, 106, 93, 103, 0, 0, 380], 'all_points_y': [412, 414, 568, 566, 516, 442, 483, 471, 437, 398, 269, 412], 'category': 0}, 'region_attributes': {}}, '1': {'shape_attributes': {'name': 'polygon', 'all_points_x': [380, 380, 331, 317, 332, 117, 106, 93, 103, 0, 0, 380], 'all_points_y': [412, 414, 568, 566, 516, 442, 483, 471, 437, 398, 269, 412], 'category': 0}, 'region_attributes': {}}}
{'file_ref': '', 'size': 3318900, 'filename': '11_20221009_111423_SV1-03_SV_RD_8bit_RGB_50cm_Rhoon_11000_13000.png', 'base64_img_data': '', 'file_attributes': {}, 'regions': {'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [380, 380, 331, 317, 332, 117, 106, 93, 10

Creating JSONs for Detectron2 on train:   6%|▌         | 1407/22719 [00:00<00:04, 4815.97it/s]

2
{'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [999, 1000, 1000, 999], 'all_points_y': [344, 335, 344, 344], 'category': 0}, 'region_attributes': {}}}
{'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [999, 1000, 1000, 999], 'all_points_y': [344, 335, 344, 344], 'category': 0}, 'region_attributes': {}}, '1': {'shape_attributes': {'name': 'polygon', 'all_points_x': [999, 1000, 1000, 999], 'all_points_y': [344, 335, 344, 344], 'category': 0}, 'region_attributes': {}}}
{'file_ref': '', 'size': 3165795, 'filename': '6_20220717_104955_SV2-01_SV_RD_8bit_RGB_50cm_Venlo_13000_6000.png', 'base64_img_data': '', 'file_attributes': {}, 'regions': {'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [999, 1000, 1000, 999], 'all_points_y': [344, 335, 344, 344], 'category': 0}, 'region_attributes': {}}, '1': {'shape_attributes': {'name': 'polygon', 'all_points_x': [999, 1000, 1000, 999], 'all_points_y': [344, 335, 344, 344], 'category': 0}, 'region_attributes'

Creating JSONs for Detectron2 on train:  12%|█▏        | 2766/22719 [00:00<00:03, 5726.14it/s]

...

2
{'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [543, 543, 562, 568, 583, 576, 583, 590, 606, 598, 560, 545, 549, 550, 566, 569, 610, 607, 567, 469, 465, 472, 543], 'all_points_y': [460, 482, 486, 476, 489, 501, 506, 496, 514, 524, 489, 490, 558, 616, 611, 579, 542, 582, 624, 622, 489, 459, 460], 'category': 0}, 'region_attributes': {}}}
{'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [543, 543, 562, 568, 583, 576, 583, 590, 606, 598, 560, 545, 549, 550, 566, 569, 610, 607, 567, 469, 465, 472, 543], 'all_points_y': [460, 482, 486, 476, 489, 501, 506, 496, 514, 524, 489, 490, 558, 616, 611, 579, 542, 582, 624, 622, 489, 459, 460], 'category': 0}, 'region_attributes': {}}, '1': {'shape_attributes': {'name': 'polygon', 'all_points_x': [543, 543, 562, 568, 583, 576, 583, 590, 606, 598, 560, 545, 549, 550, 566, 569, 610, 607, 567, 469, 465, 472, 543], 'all_points_y': [460, 482, 486, 476, 489, 501, 506, 496, 514, 524, 489, 490, 558, 616, 611, 579, 542, 582, 62

Creating JSONs for Detectron2 on train:  90%|████████▉ | 20369/22719 [00:04<00:00, 6470.74it/s]

2
{'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [767, 757, 918, 926, 952, 947, 961, 956, 966, 970, 1000, 1000, 998, 994, 946, 907, 862, 844, 759, 742, 753, 736, 715, 683, 743, 767], 'all_points_y': [521, 542, 625, 609, 625, 640, 645, 664, 667, 656, 673, 708, 707, 718, 695, 762, 733, 721, 674, 653, 625, 611, 655, 632, 508, 521], 'category': 0}, 'region_attributes': {}}}
{'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [767, 757, 918, 926, 952, 947, 961, 956, 966, 970, 1000, 1000, 998, 994, 946, 907, 862, 844, 759, 742, 753, 736, 715, 683, 743, 767], 'all_points_y': [521, 542, 625, 609, 625, 640, 645, 664, 667, 656, 673, 708, 707, 718, 695, 762, 733, 721, 674, 653, 625, 611, 655, 632, 508, 521], 'category': 0}, 'region_attributes': {}}, '1': {'shape_attributes': {'name': 'polygon', 'all_points_x': [767, 757, 918, 926, 952, 947, 961, 956, 966, 970, 1000, 1000, 998, 994, 946, 907, 862, 844, 759, 742, 753, 736, 715, 683, 743, 767], 'all_points_y': [521, 542, 6

Creating JSONs for Detectron2 on train: 100%|██████████| 22719/22719 [00:04<00:00, 5071.28it/s]

2
{'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [503, 487, 476, 506, 492, 535, 542, 569, 566, 585, 591, 615, 634, 646, 625, 622, 551, 578, 499, 488, 504, 503], 'all_points_y': [623, 630, 586, 575, 506, 492, 518, 511, 491, 481, 509, 503, 498, 552, 560, 552, 571, 670, 689, 636, 632, 623], 'category': 0}, 'region_attributes': {}}}
{'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [503, 487, 476, 506, 492, 535, 542, 569, 566, 585, 591, 615, 634, 646, 625, 622, 551, 578, 499, 488, 504, 503], 'all_points_y': [623, 630, 586, 575, 506, 492, 518, 511, 491, 481, 509, 503, 498, 552, 560, 552, 571, 670, 689, 636, 632, 623], 'category': 0}, 'region_attributes': {}}, '1': {'shape_attributes': {'name': 'polygon', 'all_points_x': [503, 487, 476, 506, 492, 535, 542, 569, 566, 585, 591, 615, 634, 646, 625, 622, 551, 578, 499, 488, 504, 503], 'all_points_y': [623, 630, 586, 575, 506, 492, 518, 511, 491, 481, 509, 503, 498, 552, 560, 552, 571, 670, 689, 636, 632, 623], 'categ

In [19]:
#Validation set
val_dict = {}

for file in tqdm(val, desc="Creating JSONs for Detectron2 on val", ncols=150, bar_format="{l_bar}{bar:10}{r_bar}"):
    file_path = os.path.join(nso_small_tiles, file)
    img_id = file.split(".tif")[0]
    geojson_path = os.path.join(nso_geojson, f"{img_id}.geojson")
    if os.path.exists(geojson_path): 
        with open(geojson_path) as f:
            gj = geojson.load(f)

        regions = {}
        num_buildings = len(gj["features"])
        if num_buildings > 0:
            gdal_image = gdal.Open(file_path)
            pixel_width, pixel_height = gdal_image.GetGeoTransform()[1], gdal_image.GetGeoTransform()[5]
            originX, originY = gdal_image.GetGeoTransform()[0], gdal_image.GetGeoTransform()[3]

            for i in range(num_buildings):
                points = gj["features"][i]["geometry"]["coordinates"][0]
                if len(points) == 1:
                    points = points[0]

                all_points_x, all_points_y = [], []
                for j in range(len(points)):
                    all_points_x.append(int(round((points[j][0] - originX) / pixel_width)))
                    all_points_y.append(int(round((points[j][1] - originY) / pixel_height)))

                regions[str(i)] = {"shape_attributes":
                                       {"name": "polygon",
                                        "all_points_x": all_points_x,
                                        "all_points_y": all_points_y,
                                        "category": 0
                                       },
                                   "region_attributes": {}
                                  }

        dictionary = {"file_ref": '',
                      "size": os.path.getsize(file_path),
                      "filename": file.replace(".tif", ".png"),
                      "base64_img_data": '',
                      "file_attributes": {},
                      "regions": regions,
                      "origin_x": originX,
                      "origin_y": originY
                     }

        val_dict[file.replace(".tif", ".png")] = dictionary

with open("../NSO/val/nso.json", "w") as f:
    json.dump(val_dict, f)

Creating JSONs for Detectron2 on val: 100%|██████████| 7573/7573 [00:01<00:00, 5875.29it/s]


In [22]:
#Test set
test_dict = {}

for file in tqdm(test, desc="Creating JSONs for Detectron2 on test", ncols=150, bar_format="{l_bar}{bar:10}{r_bar}"):
    file_path = os.path.join(nso_small_tiles, file)
    img_id = file.split(".tif")[0]
    geojson_path = os.path.join(nso_geojson, f"{img_id}.geojson")
    if os.path.exists(geojson_path): 
        with open(geojson_path) as f:
            gj = geojson.load(f)

        regions = {}
        num_buildings = len(gj["features"])
        if num_buildings > 0:
            gdal_image = gdal.Open(file_path)
            pixel_width, pixel_height = gdal_image.GetGeoTransform()[1], gdal_image.GetGeoTransform()[5]
            originX, originY = gdal_image.GetGeoTransform()[0], gdal_image.GetGeoTransform()[3]

            for i in range(num_buildings):
                points = gj["features"][i]["geometry"]["coordinates"][0]
                if len(points) == 1:
                    points = points[0]

                all_points_x, all_points_y = [], []
                for j in range(len(points)):
                    all_points_x.append(int(round((points[j][0] - originX) / pixel_width)))
                    all_points_y.append(int(round((points[j][1] - originY) / pixel_height)))

                regions[str(i)] = {"shape_attributes":
                                       {"name": "polygon",
                                        "all_points_x": all_points_x,
                                        "all_points_y": all_points_y,
                                        "category": 0
                                       },
                                   "region_attributes": {}
                                  }

        dictionary = {"file_ref": '',
                      "size": os.path.getsize(file_path),
                      "filename": file.replace(".tif", ".png"),
                      "base64_img_data": '',
                      "file_attributes": {},
                      "regions": regions,
                      "origin_x": originX,
                      "origin_y": originY
                     }

        test_dict[file.replace(".tif", ".png")] = dictionary

with open("../NSO/test/nso.json", "w") as f:
    json.dump(test_dict, f)

Creating JSONs for Detectron2 on test: 100%|██████████| 7574/7574 [00:01<00:00, 5931.06it/s]


In [20]:
# Create single via_region_data training dataset => can be skipped if only one .json file.=>But then change file name
jsons = ["../NSO/train/nso.json"]

result = {}
for file in jsons:
    with open(file, "r") as f:
        loaded = json.load(f)
        
    #https://realpython.com/iterate-through-dictionary-python/
    for key, value in loaded.items():
        result[key] = value

with open("../NSO/train/via_region_data.json", "w") as file:
    json.dump(result, file)

In [21]:
# Create via_region JSON for entire validation dataset => can be skipped if only one .json file.=>But then change file name
jsons = ["../NSO/val/nso.json"]

result = {}
for file in jsons:
    with open(file, "r") as f:
        loaded = json.load(f)
    for key, value in loaded.items():
        result[key] = value

with open("../NSO/val/via_region_data.json", "w") as file:
    json.dump(result, file)

print("Done creating JSONs")


Done creating JSONs


In [23]:
# Create via_region JSON for entire test dataset => can be skipped if only one .json file.=>But then change file name
jsons = ["../NSO/test/nso.json"]

result = {}
for file in jsons:
    with open(file, "r") as f:
        loaded = json.load(f)
    for key, value in loaded.items():
        result[key] = value

with open("../NSO/test/via_region_data.json", "w") as file:
    json.dump(result, file)

print("Done creating JSONs")

Done creating JSONs
